In [ ]:
from elasticsearch import Elasticsearch 
import json
import nltk
from nltk.tokenize import word_tokenize
import itertools
from collections import Counter
import string

In [ ]:
es=Elasticsearch([{'host':'127.0.0.1','port':9200}])

In [ ]:
def search_by_screen_name(index, name, es):
    res= es.search(index=index,body=
    {
      "query": {
        "match": {
          "screen_name": name
        }
      }
    })
    return res

In [ ]:
res = es.search(
            index=config['elasticsearch_indices']['usertweets']['name'],
            body={'query': {'match': {'user.screen_name': username}}}
        )

        hits = res['hits']['hits']

        body = {
            # data extracted from tweets
            'full_text': [r['_source']['full_text'] for r in hits],
            'hashtags': [[h['text']
                    for h in r['_source']['entities']['hashtags']]
                    for r in hits if r['_source']['entities']['hashtags']],
            # basic user data
            'location': hits[0]['_source']['user']['location'],
            'description': hits[0]['_source']['user']['description'],
            'screen_name': username,
            'name': hits[0]['_source']['user']['name']
        }

In [ ]:
res = search_by_screen_name("users", "sterling7", es)

In [ ]:
def flat_list(l):
    return  [item for sublist in l for item in sublist]

def common_tokens(tokens, n=20):
    sentences = (list(itertools.chain(tokens)))
    flat_sentences = flat_list(sentences)
    counts = Counter(flat_sentences)
    
    return counts

In [ ]:
list_tokens=[]
stop_words=nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

for text in (res["hits"]["hits"][0]["_source"]["full_text"]):
    tokens_clean = []
    for word in word_tokenize(text):
        if word.lower() not in stop_words and word.lower() not in punctuation and not word.isnumeric() and len(word)> 1:
            tokens_clean.append(word.lower())
    list_tokens.append(tokens_clean)

In [ ]:
counts = common_tokens(list_tokens)

In [ ]:
[k for k,v in counts.most_common(30)]

In [ ]:
counts.most_common(30)

In [ ]:
hashtags = res["hits"]["hits"][0]["_source"]["hashtags"]

In [ ]:
hashcounts = common_tokens(hashtags)

In [ ]:
hashcounts.most_common(30)

In [ ]:
[k for k,v in hashcounts.most_common(30)]